In [22]:
#import modules
import lxml.etree as etree
import gensim
import spacy
import glob
import copy

In [2]:
hbbw_xmls = glob.glob('/home/phillip/Documents/uzh/phd/bullinger/bullinger-korpus/data/editions/hbbw/*/*.xml')
vbs_xmls = glob.glob('/home/phillip/Documents/uzh/phd/bullinger/bullinger-korpus/data/editions/vbs/*/*.xml')
transcribed_xmls = glob.glob('/home/phillip/Documents/uzh/phd/bullinger/bullinger-korpus/data/non_edited/transcribed/*.xml')

In [3]:
all_xmls = hbbw_xmls + vbs_xmls + transcribed_xmls
len(all_xmls)

8640

In [15]:
nlp = spacy.load('la_core_web_lg')

In [30]:
texts = []
for i, xml in enumerate(all_xmls):
    if i % 100 == 0:
        print('I am at file {}'.format(i))
    text = []
    la_sents = 0
    xml_open = etree.parse(open(xml, 'r'))
    xml_name = xml.split('/')[-1].replace('.xml', '') 
    root = xml_open.getroot()
    sentences = root.findall('.//s')
    for s in sentences:
        if s.get('lang') == 'la':
            s_copy = copy.deepcopy(s)
            for fl in s_copy.findall('.//fl'):
                fl.clear()
                try:
                    s_copy.remove(fl)
                except:
                    s_copy == s_copy
            
            la_sents += 1
            try:
                doc = nlp("".join(s_copy.itertext()).replace('[', '').replace(']', ''))
                lemmas = [w.lemma_ for w in doc if w.pos_ in ['NOUN', 'PROPN']]
                text += lemmas
            except TypeError:
                continue
    if la_sents >= 5:
        texts.append([{xml_name: text}])

I am at file 0
I am at file 100
I am at file 200
I am at file 300
I am at file 400
I am at file 500
I am at file 600
I am at file 700
I am at file 800
I am at file 900
I am at file 1000
I am at file 1100
I am at file 1200
I am at file 1300
I am at file 1400
I am at file 1500
I am at file 1600
I am at file 1700
I am at file 1800
I am at file 1900
I am at file 2000
I am at file 2100
I am at file 2200
I am at file 2300
I am at file 2400
I am at file 2500
I am at file 2600
I am at file 2700
I am at file 2800
I am at file 2900
I am at file 3000
I am at file 3100
I am at file 3200
I am at file 3300
I am at file 3400
I am at file 3500
I am at file 3600
I am at file 3700
I am at file 3800
I am at file 3900
I am at file 4000
I am at file 4100
I am at file 4200
I am at file 4300
I am at file 4400
I am at file 4500
I am at file 4600
I am at file 4700
I am at file 4800
I am at file 4900
I am at file 5000
I am at file 5100
I am at file 5200
I am at file 5300
I am at file 5400
I am at file 5500
I am

In [31]:
len(texts)

7374

In [32]:
import pickle

In [33]:
# texts all in a list, in the form of [[{file_name: ['tok1', 'tok2', ...]}], [{file_name: ['tok1', 'tok2', ...]}], ...]
with open('bullinger-letters-tok-lem.list', 'wb') as file:
    pickle.dump(texts, file)

In [37]:
values = [value_list for inner_list in texts for dictionary in inner_list for value_list in dictionary.values()]

In [40]:
dictionary = gensim.corpora.Dictionary(values)
corpus = [dictionary.doc2bow(doc) for doc in values]

In [42]:
# Train LDA model
num_topics = 50  # You can adjust this based on your data and requirements
lda = gensim.models.LdaModel(corpus, id2word=dictionary, num_topics=num_topics, passes=15)

# Print the topics
for idx, topic in lda.print_topics(-1):
    print(f"Topic: {idx} \nWords: {topic}\n")

Topic: 0 
Words: 0.035*"dominus" + 0.031*"urbs" + 0.028*"Uadianus" + 0.026*"Caesar" + 0.023*"Sangallus" + 0.023*"annus" + 0.022*"dies" + 0.022*"episcopus" + 0.021*"amicus" + 0.020*"Ioachimus"

Topic: 1 
Words: 0.056*"litera" + 0.030*"filius" + 0.025*"dies" + 0.025*"rex" + 0.019*"dominus" + 0.017*"mensis" + 0.015*"amicus" + 0.014*"princeps" + 0.013*"uxor" + 0.012*"Tiguri"

Topic: 2 
Words: 0.021*"Rogandus" + 0.014*"Do" + 0.012*"dan" + 0.012*"corrector" + 0.011*"capitulus" + 0.010*"mona" + 0.009*"foeminus" + 0.009*"hyemis" + 0.009*"hortus" + 0.008*"tedium"

Topic: 3 
Words: 0.022*"Gwaltherus" + 0.010*"Jacobus" + 0.010*"conventiculum" + 0.008*"uictorię" + 0.004*"Observandissimo" + 0.004*"insipientia" + 0.004*"Ratisponensibus" + 0.003*"fedus" + 0.003*"forsitan" + 0.003*"Zender"

Topic: 4 
Words: 0.110*"Curię" + 0.016*"Balthasar" + 0.015*"Henryco" + 0.015*"Cęsaris" + 0.014*"Corsica" + 0.011*"Audies" + 0.011*"Satanę" + 0.010*"praestantissimum" + 0.009*"Sulcero" + 0.009*"Brugensis"

Topic: 5 

In [43]:
lda.save("gensim_lda_model-bullinger_nouns_50topics")

In [44]:
# Train LDA model with Mallet
mallet_path = '/home/phillip/Documents/applications/Mallet/bin/mallet' 

num_topics = 50  # You can adjust this based on your data and requirements
lda_mallet = gensim.models.wrappers.LdaMallet(corpus, id2word=dictionary, num_topics=num_topics)

# Print the topics
for idx, topic in lda_mallet.print_topics(-1):
    print(f"Topic: {idx} \nWords: {topic}\n")

AttributeError: module 'gensim.models' has no attribute 'wrappers'